In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

In [3]:
df = pd.read_parquet('/home/brandon/MLOps Zoomcamp/yellow_tripdata_2022-01.parquet')

In [4]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


In [5]:
shape = df.shape
print(f'The shape of the dataframe and the number of columns is: {shape}')

The shape of the dataframe and the number of columns is: (2463931, 19)


In [6]:
df.tpep_pickup_datetime = pd.to_datetime(df['tpep_pickup_datetime'])
df.tpep_dropoff_datetime = pd.to_datetime(df['tpep_dropoff_datetime'])

In [7]:
df['duration'] = df.tpep_dropoff_datetime -df.tpep_pickup_datetime

In [8]:
td = df.duration.iloc[0]
td.total_seconds() / 60

17.816666666666666

In [9]:
duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [10]:
df.duration.describe(percentiles=[0.95, 0.98, 0.99])

count                      2463931
mean     0 days 00:14:12.732175129
std      0 days 00:46:26.718308266
min              -3 days +14:37:36
50%                0 days 00:10:11
95%                0 days 00:31:56
98%                0 days 00:42:09
99%                0 days 00:50:51
max                5 days 21:53:11
Name: duration, dtype: object

In [11]:
# Remove outliers
df = df[duration <= 60]
print(f'The shape of the dataframe and the number of columns is: {df.shape}')

#Fraction of records left after removing outliers
df.shape[0] / shape[0]

The shape of the dataframe and the number of columns is: (2451443, 20)


0.9949316762522976

In [12]:
#Create Categorical and Numerical columns
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

#Convert to string
df[categorical] = df[categorical].astype('str')

#Find the categorical dtypes
df[categorical].dtypes

#Create Feature Matrix
X = df[categorical + numerical].to_dict(orient='records')






/tmp/ipykernel_24321/1059614023.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype('str')


In [13]:
#Take a sample of the data
X = X[:20000]

In [14]:
#Print the first 5 samples
X[:5]

[{'PULocationID': '142', 'DOLocationID': '236', 'trip_distance': 3.8},
 {'PULocationID': '236', 'DOLocationID': '42', 'trip_distance': 2.1},
 {'PULocationID': '166', 'DOLocationID': '166', 'trip_distance': 0.97},
 {'PULocationID': '114', 'DOLocationID': '68', 'trip_distance': 1.09},
 {'PULocationID': '68', 'DOLocationID': '163', 'trip_distance': 4.3}]

In [15]:
#Fit the DictVectorizer
dv = DictVectorizer(sparse=False)
dv.fit(X)

#Transform the Feature Matrix
X = dv.transform(X)

#Get Feature Names
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=11',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',
 'DOLocationID=146',
 'DOLocationID=147',
 'DOLocationID=148',
 'DOLocationID=149',
 'DOLocationID=15',


In [16]:
#Put all previous code into a function
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df['tpep_dropoff_datetime'])
    df.tpep_pickup_datetime = pd.to_datetime(df['tpep_pickup_datetime'])
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 0) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype('str')
    
    return df

In [17]:
df_train = read_dataframe('/home/brandon/MLOps Zoomcamp/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('/home/brandon/MLOps Zoomcamp/yellow_tripdata_2022-02.parquet')

In [18]:
#Take Sample of the data
df_train = df_train.sample(n=20000, random_state=1)
df_val = df_val.sample(n=20000, random_state=1)

In [19]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
x_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
x_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

#Train the model
lr = LinearRegression()
lr.fit(x_train, y_train)

#Predict on train
y_pred_train = lr.predict(x_train)

#Calculate the RMSE
y_pred_val = lr.predict(x_val)

#Print the RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))

In [22]:
rmse_train


5.042333081718575

In [23]:
rmse_val

2365016114.328127